# viz-3a-iris-classification.ipynb

This notebook demonstrates how small multiples of stacked bar chart histograms can be used to evaluate machine learning results, in this case comparing original values to predicted categorizations.

In [1]:
import datetime
import altair as alt
import eland as ed
import json
import numpy as np
import matplotlib.pyplot as plt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
from vega_datasets import data
pd_df = data.iris()
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepalLength    150 non-null float64
sepalWidth     150 non-null float64
petalLength    150 non-null float64
petalWidth     150 non-null float64
species        150 non-null object
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


The next cell takes the iris dataset from VEGA's examples and pushes it to an Elasticsearch index using `eland`.

In [3]:
ed_df = ed.pandas_to_eland(
    pd_df.dropna(),
    'localhost',
    'eland_iris',
    es_if_exists="replace",
    es_refresh=True
)
ed_df.columns

Index(['petalLength', 'petalWidth', 'sepalLength', 'sepalWidth', 'species'], dtype='object')

Before you can run the next cell, you need to run a Machine Learning job doing classification in Elasticsearch using the following configuration. It will write the results to an index `iris_classification_3` which we can then access in the following cell.

```json
{
  "id": "iris_classification_3",
  "create_time": 1604478053294,
  "version": "8.0.0",
  "description": "",
  "source": {
    "index": [
      "eland_iris"
    ],
    "query": {
      "match_all": {}
    }
  },
  "dest": {
    "index": "iris_classification_3",
    "results_field": "ml"
  },
  "analysis": {
    "classification": {
      "dependent_variable": "species",
      "num_top_feature_importance_values": 10,
      "class_assignment_objective": "maximize_minimum_recall",
      "num_top_classes": 2,
      "prediction_field_name": "species_prediction",
      "training_percent": 80,
      "randomize_seed": -2245814554146464500
    }
  },
  "analyzed_fields": {
    "includes": [],
    "excludes": []
  },
  "model_memory_limit": "100mb",
  "allow_lazy_start": false,
  "max_num_threads": 1
}
```

In [4]:
index_name='iris_classification_3'
ed_df = ed.DataFrame('localhost:9200', index_name)
ed_df.head()

,ml.feature_importance,ml.feature_importance.classes,ml.feature_importance.classes.class_name,ml.feature_importance.classes.importance,ml.feature_importance.feature_name,ml.feature_importance.importance,ml.is_training,ml.prediction_probability,ml.prediction_score,ml.species_prediction,ml.top_classes,ml.top_classes.class_name,ml.top_classes.class_probability,ml.top_classes.class_score,ml__incremental_id,petalLength,petalWidth,sepalLength,sepalWidth,species
0,"[{'feature_name': 'petalLength', 'classes': [{...",NaN,NaN,NaN,NaN,NaN,True,0.991117,0.83755,setosa,"[{'class_probability': 0.9911168670896968, 'cl...",NaN,NaN,NaN,0,1.4,0.2,5.1,3.5,setosa
1,"[{'feature_name': 'petalLength', 'classes': [{...",NaN,NaN,NaN,NaN,NaN,True,0.991117,0.83755,setosa,"[{'class_probability': 0.9911168670896968, 'cl...",NaN,NaN,NaN,1,1.4,0.2,4.9,3.0,setosa
2,"[{'feature_name': 'petalLength', 'classes': [{...",NaN,NaN,NaN,NaN,NaN,True,0.991117,0.83755,setosa,"[{'class_probability': 0.9911168670896968, 'cl...",NaN,NaN,NaN,2,1.3,0.2,4.7,3.2,setosa
4,"[{'feature_name': 'petalLength', 'classes': [{...",NaN,NaN,NaN,NaN,NaN,True,0.991117,0.83755,setosa,"[{'class_probability': 0.9911168670896968, 'cl...",NaN,NaN,NaN,4,1.4,0.2,5.0,3.6,setosa
6,"[{'feature_name': 'petalLength', 'classes': [{...",NaN,NaN,NaN,NaN,NaN,True,0.991117,0.83755,setosa,"[{'class_probability': 0.9911168670896968, 'cl...",NaN,NaN,NaN,6,1.4,0.3,4.6,3.4,setosa


In [5]:
url = 'http://localhost:9200/'+index_name+'/_search?size=1000'
url_data = alt.Data(url=url, format=alt.DataFormat(property='hits.hits',type='json'))

fields = ['petalLength', 'petalWidth', 'sepalLength', 'sepalWidth']

rename_dict = dict((a, 'datum._source.'+a) for a in fields)

def small_multiples(split='species'):
    url_chart = alt.Chart(url_data).transform_calculate(
        species_prediction='datum._source.ml.species_prediction',
        species='datum._source.species'
    ).transform_calculate(**rename_dict).transform_fold(
        fields,
        as_=['attribute', 'value']
    ).mark_bar().encode(
        alt.X('value:Q', title='', bin=True),
        alt.Y('count()', title=''),
        tooltip=[
            alt.Tooltip('value:Q', bin=True, title='x'),
            alt.Tooltip('count()', title='y')
        ],
        color=split+':N'
    ).properties(
        width=150,
        height=150
    )

    url_charts = alt.ConcatChart(
        concat=[
          url_chart.transform_filter(alt.datum.attribute == attribute).properties(title=attribute)
          for attribute in sorted(['petalLength', 'petalWidth', 'sepalLength', 'sepalWidth'])
        ],
        columns=2
    ).resolve_axis(
        x='independent',
        y='independent'
    ).resolve_scale(
        x='independent', 
        y='independent'
    )
    
    return url_charts

chart_raw = small_multiples('species')
chart_raw

alt.ConcatChart(...)

In [6]:
chart_prediction = small_multiples('species_prediction')
chart_prediction

alt.ConcatChart(...)

In [7]:
alt.Chart(url_data).transform_calculate(
    species_prediction='datum._source.ml.species_prediction',
    species='datum._source.species'
).transform_calculate(**rename_dict).mark_circle(size=30).encode(
    x='petalLength:Q',
    y='petalWidth:Q',
    color='species:N',
    tooltip=['species:N']
)

alt.Chart(...)

In [8]:
from kibana_vega_util import saveVegaLiteVis
from elasticsearch import Elasticsearch 
es=Elasticsearch([{'host':'localhost','port':9200}])

saveVegaLiteVis(es, index_name, 'iris-histogram-raw', chart_raw, resultSize=10000, timeField=False)
saveVegaLiteVis(es, index_name, 'iris-histogram-prediction', chart_prediction, resultSize=10000, timeField=False)

{'_index': '.kibana_3',
 '_id': 'visualization:iris-histogram-prediction',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 423,
 '_primary_term': 2}